In [1]:
import xarray as xr
import pandas as pd
from pathlib import Path

data_dir = "./GFED5/daily/" 
start_date = "2018-04"
end_date = "2020-12" 
aggregation = "mean"

# Convert dates to datetime for comparison
start_date = pd.to_datetime(start_date, format="%Y-%m")
end_date = pd.to_datetime(end_date, format="%Y-%m")

# List all NetCDF files in the directory

all_files = sorted(Path(data_dir).glob("*.nc"))

# Filter files by date range

filtered_files = [
     str(file) for file in all_files
     if start_date <= pd.to_datetime(file.stem[-6:], format="%Y%m") <= end_date
     ]

if not filtered_files:
     raise ValueError("No files match the specified date range.")

In [2]:
ds = xr.open_mfdataset(filtered_files)

In [3]:
carbon_mean = ds["C"].mean(dim="time")

In [4]:
carbon_mean

<xarray.DataArray 'C' (lat: 720, lon: 1440)> Size: 4MB
dask.array<mean_agg-aggregate, shape=(720, 1440), dtype=float32, chunksize=(720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 3kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9

In [5]:
data = carbon_mean.to_dataframe().reset_index()

In [6]:
import pydeck as pdk

layer = pdk.Layer(
     "HexagonLayer",
     data,
     get_position=["lon", "lat"],
     auto_highlight=True,
     elevation_scale=50,
     pickable=True,
     extruded=True,
     coverage=1,
     radius=1000
)

view_state = pdk.ViewState(
     longitude=0,
     latitude=0,
     zoom=1,
     min_zoom=0,
     max_zoom=15,
     pitch=40.5,
     bearing=-27.396674584323023,
)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("hexagon.html", notebook_display=False)

In [7]:
# heatmap

layer = pdk.Layer(
     "HeatmapLayer",
     data,
     get_position=["lon", "lat"],
     get_weight="C",
)

view_state = pdk.ViewState(
     longitude=0,
     latitude=0,
     zoom=1,
     min_zoom=0,
     max_zoom=15,
     pitch=40.5,
     bearing=-27.396674584323023,
)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)

r.to_html("heatmap.html", notebook_display=False)